<a href="https://colab.research.google.com/github/Manpreet102005/Diabetes-Prediction/blob/main/Diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
 pip install opendatasets

In [3]:
import opendatasets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import os

In [4]:
opendatasets.download("https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: manpreetsinghkashyap
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database


100%|██████████| 8.91k/8.91k [00:00<00:00, 34.6MB/s]

In [5]:
df=pd.read_csv("/content/pima-indians-diabetes-database/diabetes.csv")

In [6]:
from sklearn.model_selection import train_test_split
train_df_1,test_df=train_test_split(df,test_size=0.2,random_state=758)
train_df,val_df=train_test_split(train_df_1,test_size=0.25,random_state=758)

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 460 entries, 395 to 727
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               460 non-null    int64  
 1   Glucose                   460 non-null    int64  
 2   BloodPressure             460 non-null    int64  
 3   SkinThickness             460 non-null    int64  
 4   Insulin                   460 non-null    int64  
 5   BMI                       460 non-null    float64
 6   DiabetesPedigreeFunction  460 non-null    float64
 7   Age                       460 non-null    int64  
 8   Outcome                   460 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 35.9 KB


In [8]:
train_df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,460.000000,460.00000,460.000000,460.000000,460.000000,460.00000,460.000000,460.000000,460.000000
mean,3.734783,121.48913,68.469565,20.810870,83.295652,32.15000,0.478467,32.878261,0.367391
std,3.421591,32.58055,19.260205,16.320359,120.664733,7.85624,0.342261,11.853240,0.482619
min,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.084000,21.000000,0.000000
25%,1.000000,97.00000,62.000000,0.000000,0.000000,27.40000,0.239500,24.000000,0.000000
50%,3.000000,118.50000,72.000000,23.000000,36.500000,32.40000,0.373500,29.000000,0.000000
75%,6.000000,143.00000,80.000000,33.000000,126.000000,36.60000,0.641250,41.000000,1.000000
max,17.000000,199.00000,114.000000,99.000000,744.000000,59.40000,2.420000,81.000000,1.000000


In [9]:
df.isna().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


In [10]:
inputs=list(df.columns[:-1])
target="Outcome"

In [11]:
X_train=train_df[inputs].copy()
Y_train=train_df[target].copy()
X_val=val_df[inputs].copy()
Y_val=val_df[target].copy()
X_test=test_df[inputs].copy()
Y_test=test_df[target].copy()

In [12]:
numeric_cols=list(X_train.select_dtypes('number'))

In [13]:
# Scaling of inputs
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler().fit(X_train,Y_train)
X_train[numeric_cols]=scaler.transform(X_train[numeric_cols])
X_val[numeric_cols]=scaler.transform(X_val[numeric_cols])
X_test[numeric_cols]=scaler.transform(X_test[numeric_cols])

In [14]:
X_train.corr()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Pregnancies,1.000000,0.123000,0.189838,-0.061880,-0.024996,0.016315,-0.071887,0.580165
Glucose,0.123000,1.000000,0.146481,0.091438,0.334260,0.284250,0.179688,0.222987
BloodPressure,0.189838,0.146481,1.000000,0.194712,0.098417,0.310252,0.031197,0.211325
SkinThickness,-0.061880,0.091438,0.194712,1.000000,0.439563,0.383551,0.218880,-0.085329
Insulin,-0.024996,0.334260,0.098417,0.439563,1.000000,0.210639,0.262018,-0.038414
BMI,0.016315,0.284250,0.310252,0.383551,0.210639,1.000000,0.153818,0.008408
DiabetesPedigreeFunction,-0.071887,0.179688,0.031197,0.218880,0.262018,0.153818,1.000000,0.033208
Age,0.580165,0.222987,0.211325,-0.085329,-0.038414,0.008408,0.033208,1.000000


Base Models

Hyperparameter tuning using Grid Search

In [15]:
def grid_search(model,space):
  from sklearn.model_selection import GridSearchCV, StratifiedKFold
  cv=StratifiedKFold(n_splits=10,random_state=756,shuffle=True)
  result=GridSearchCV(model,space,scoring='accuracy',n_jobs=-1,cv=cv).fit(X_train,Y_train)
  print(f"Best Parameters: {result.best_params_}")
  print(f'Best Scores : {result.best_score_}')

Logistic Regression


In [28]:
# logistic regression
from sklearn.linear_model import LogisticRegression
logistic_regression=LogisticRegression(random_state=756,).fit(X_train,Y_train)


In [29]:
def check_score(model):
  from sklearn.metrics import accuracy_score
  print(f"Model: {model}")
  print(f"Train Score: {accuracy_score(Y_train,model.predict(X_train))} | Validation Score: {accuracy_score(Y_val,model.predict(X_val))}")


In [30]:
check_score(logistic_regression)

Model: LogisticRegression(random_state=756)
Train Score: 0.7782608695652173 | Validation Score: 0.7922077922077922


In [31]:
space={
    'tol':[1e-6,1e-5,10-4],
    'max_iter':[1000,2000,3000],
    'warm_start':[False,True],
    'solver':['lbfgs','liblinear']
}

In [32]:
grid_search(logistic_regression,space)

Best Parameters: {'max_iter': 1000, 'solver': 'liblinear', 'tol': 1e-06, 'warm_start': False}
Best Scores : 0.7695652173913043


In [33]:
logistic_regression=LogisticRegression(random_state=756,max_iter=1000,solver='liblinear',tol=1e-06).fit(X_train,Y_train)


In [34]:
check_score(logistic_regression)

Model: LogisticRegression(max_iter=1000, random_state=756, solver='liblinear',
                   tol=1e-06)
Train Score: 0.7760869565217391 | Validation Score: 0.7987012987012987


Ridge Classifier

In [40]:
from sklearn.linear_model import RidgeClassifier
ridge=RidgeClassifier(random_state=756).fit(X_train,Y_train)

In [41]:
check_score(ridge)

Model: RidgeClassifier(random_state=756)
Train Score: 0.7891304347826087 | Validation Score: 0.7857142857142857


In [45]:
space={
    'alpha':[0.1,0.2,0.5,1.0,1.5,2.0,2.5],
    'max_iter':[1000,2000,3000],
    'tol':[1e-6,1e-5,10-4],
    'solver':['svd','cholesky','lsqr','sparse_cg','sag','saga']
}
grid_search(ridge,space)

Best Parameters: {'alpha': 0.5, 'max_iter': 1000, 'solver': 'svd', 'tol': 1e-06}
Best Scores : 0.7891304347826088


In [43]:
ridge=RidgeClassifier(alpha=0.5,random_state=756,max_iter=1000,tol=1e-06,solver='svd').fit(X_train,Y_train)

In [44]:
check_score(ridge)

Model: RidgeClassifier(alpha=0.5, max_iter=1000, random_state=756, solver='svd',
                tol=1e-06)
Train Score: 0.7891304347826087 | Validation Score: 0.7792207792207793


No change in scores with changing hyperparameters of Ridge Classifier

Decision Tree

In [48]:
from sklearn.tree import DecisionTreeClassifier
decision_tree=DecisionTreeClassifier(random_state=756).fit(X_train,Y_train)

In [49]:
check_score(decision_tree)

Model: DecisionTreeClassifier(random_state=756)
Train Score: 1.0 | Validation Score: 0.6493506493506493


In [51]:
space={
    'max_depth':[None,1,2,3,4,5,6,7,8,9,10],
    'min_samples_split':[2,3,4,5,6,7,8,9,10],
    'min_samples_leaf':[1,2,3,4,5,6,7,8,9,10],
    'max_features':[3,4,5,6,7,8,9]
}
grid_search(decision_tree,space)

Best Parameters: {'max_depth': None, 'max_features': 6, 'min_samples_leaf': 10, 'min_samples_split': 2}
Best Scores : 0.7717391304347826


In [52]:
decision_tree=DecisionTreeClassifier(random_state=756,max_features=6,min_samples_leaf=10,min_samples_split=2).fit(X_train,Y_train)

In [53]:
check_score(decision_tree)

Model: DecisionTreeClassifier(max_features=6, min_samples_leaf=10, random_state=756)
Train Score: 0.8608695652173913 | Validation Score: 0.6883116883116883


In [54]:
!git config --global user.email "singhmanpreet102005@gmail.com"
!git config --global user.name "Manpreet102005"


In [55]:
!git clone https://github.com/Manpreet102005/Diabetes-Prediction


Cloning into 'Diabetes-Prediction'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (4/4), done.


In [56]:
%cd /content/Diabetes-Prediction
!git add .
!git commit -m "First Commitment"


/content/Diabetes-Prediction
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [57]:
!git push origin main


fatal: could not read Username for 'https://github.com': No such device or address
